# Init BQ Auth

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')
project_id='your-project-id' #@param{type:"string"}
%load_ext google.colab.data_table

Authenticated


# egress data usage by day, Bigquery SDK
## Last day and domain

In [ ]:
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

df = client.query('''
SELECT
  EXTRACT (DATE
  FROM
    timestamp AT TIME ZONE "Asia/Shanghai") AS Date,
    NET.HOST(httpRequest.requestUrl) as Domain,
  SUM(httpRequest.responseSize)/1024/1024/1024 AS response_GB
FROM
  `ds_us.requests`
WHERE ((httpRequest.userAgent not like '%Cloud-CDN-Google%') or (httpRequest.userAgent is NULL))
and EXTRACT(DATE FROM timestamp AT TIME ZONE "Asia/Shanghai") = DATE_ADD(CURRENT_DATE('Asia/Shanghai'), INTERVAL -1 DAY)
GROUP BY
  Date, Domain
ORDER BY
  Date ASC, response_GB DESC
''' ).to_dataframe()
df.head(20)

,Date,Domain,response_GB
0,2020-07-27,cdn-2020-t2.wshareit.com,30.15562


# CDN bandwidth usage calculation

In [ ]:
%%bigquery --project {project_id}
WITH
  bandwidth_sec AS (
  SELECT
    TIMESTAMP_TRUNC(timestamp, DAY) AS Date,
    TIMESTAMP_TRUNC(timestamp, SECOND) AS Time,
    SUM(httpRequest.responseSize)*8 AS bps
  FROM
    `ds_us.requests` as requests
    WHERE
	(((requests.timestamp ) >= ((TIMESTAMP(FORMAT_TIMESTAMP('%F %H:%M:%E*S', TIMESTAMP_ADD(TIMESTAMP_TRUNC(TIMESTAMP(FORMAT_TIMESTAMP('%F %H:%M:%E*S', CURRENT_TIMESTAMP(), 'Asia/Shanghai')), DAY), INTERVAL -5 DAY)), 'Asia/Shanghai'))) AND (requests.timestamp ) < ((TIMESTAMP(FORMAT_TIMESTAMP('%F %H:%M:%E*S', TIMESTAMP_ADD(TIMESTAMP_ADD(TIMESTAMP_TRUNC(TIMESTAMP(FORMAT_TIMESTAMP('%F %H:%M:%E*S', CURRENT_TIMESTAMP(), 'Asia/Shanghai')), DAY), INTERVAL -5 DAY), INTERVAL 5 DAY)), 'Asia/Shanghai')))))
  GROUP BY
    Date,
    Time
  ORDER BY
    Date,
    Time ASC)
SELECT
  Date,
  AVG(Mbps) AS Mbps
FROM (
  SELECT
    Date,
    (NTH_VALUE(bps, 15) OVER (PARTITION BY Date ORDER BY bps DESC ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING))/1024/1024 AS Mbps
  FROM (
    SELECT
      Date,
      Time,
      bps,
      MOD(UNIX_SECONDS(Time), 300) AS sec_counter
    FROM
      bandwidth_sec)
  WHERE
    sec_counter=0
  ORDER BY
    Date,
    bps DESC)
GROUP BY
  Date
ORDER BY
  Date ASC

,Date,Mbps
0,2020-07-09 00:00:00+00:00,NaN
1,2020-07-10 00:00:00+00:00,NaN
2,2020-07-11 00:00:00+00:00,16.538597


# egress data usage by day, Bigquery SDK
## Last day and domain

In [ ]:
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

df = client.query('''
SELECT
  EXTRACT (DATE
  FROM
    timestamp AT TIME ZONE "Asia/Shanghai") AS Date,
    NET.HOST(httpRequest.requestUrl) as Domain,
  SUM(httpRequest.responseSize)/1024/1024/1024 AS response_GB
FROM
  `ds_us.requests`
WHERE ((httpRequest.userAgent not like '%Cloud-CDN-Google%') or (httpRequest.userAgent is NULL))
and EXTRACT(DATE FROM timestamp AT TIME ZONE "Asia/Shanghai") = DATE_ADD(CURRENT_DATE('Asia/Shanghai'), INTERVAL -1 DAY)
GROUP BY
  Date, Domain
ORDER BY
  Date ASC, response_GB DESC
''' ).to_dataframe()
df.head(20)

,Date,Domain,response_GB
0,2020-08-13,34.120.250.131,0.000301
1,2020-08-13,127.0.0.1,0.000001
2,2020-08-13,185.172.110.185,0.000001


# Hit PoP Information

In [ ]:
%%bigquery --project $project_id
with tmp_table as (SELECT
  EXTRACT (DATE
  FROM
    timestamp) AS Date, jsonpayload_type_loadbalancerlogentry.cacheid AS location_id, count(*) as location_count
FROM
  `ds_us.requests` as requests
  WHERE
	(((requests.timestamp ) >= ((TIMESTAMP(FORMAT_TIMESTAMP('%F %H:%M:%E*S', TIMESTAMP_ADD(TIMESTAMP_TRUNC(TIMESTAMP(FORMAT_TIMESTAMP('%F %H:%M:%E*S', CURRENT_TIMESTAMP(), 'Asia/Shanghai')), DAY), INTERVAL -7 DAY)), 'Asia/Shanghai'))) AND (requests.timestamp ) < ((TIMESTAMP(FORMAT_TIMESTAMP('%F %H:%M:%E*S', TIMESTAMP_ADD(TIMESTAMP_ADD(TIMESTAMP_TRUNC(TIMESTAMP(FORMAT_TIMESTAMP('%F %H:%M:%E*S', CURRENT_TIMESTAMP(), 'Asia/Shanghai')), DAY), INTERVAL -7 DAY), INTERVAL 7 DAY)), 'Asia/Shanghai')))))
GROUP BY
  Date, location_id
ORDER BY
  Date ASC, location_count Desc
)
select Date, location_id, location_count
from tmp_table
where location_count > 10000

,Date,location_id,location_count
0,2020-07-11,SIN-b57fb7cd,18729
1,2020-07-11,JNB-b7f7e32d,15024
2,2020-07-11,SIN-63b91979,11563


# cache fill data usage by day

In [ ]:
%%bigquery --project $project_id
SELECT
  EXTRACT (DATE
  FROM
    timestamp AT TIME ZONE "Asia/Shanghai") AS Date,
  SUM(httpRequest.cacheFillBytes)/1024/1024/1024 AS response_GB
FROM
  `ds_us.requests`
WHERE httpRequest.cacheHit = true
GROUP BY
  Date
ORDER BY
  Date ASC

,Date,response_GB
0,2020-07-09,NaN
1,2020-07-11,9.568862
2,2020-07-12,16.722373
3,2020-07-13,26.697866
4,2020-07-14,22.642113


# cache hit egress data usage by day

In [ ]:
%%bigquery --project $project_id
SELECT
  EXTRACT (DATE
  FROM
    timestamp AT TIME ZONE "Asia/Shanghai") AS Date,
  SUM(httpRequest.responseSize)/1024/1024/1024 AS response_GB
FROM
  `ds_us.requests`
WHERE httpRequest.cacheHit=true
GROUP BY
  Date
ORDER BY
  Date ASC

,Date,response_GB
0,2020-07-09,0.000045
1,2020-07-11,21.223895
2,2020-07-12,54.092935
3,2020-07-13,58.702410
4,2020-07-14,58.503211


# Distinct URL numbers

In [ ]:
%%bigquery --project $project_id
SELECT
  count(DISTINCT(httpRequest.requestUrl))
FROM
  `ds_us.requests`

# Distinct URL numbers by day

In [ ]:
%%bigquery --project $project_id
SELECT
  EXTRACT (DATE
  FROM
    timestamp) AS Date,
  COUNT(DISTINCT(httpRequest.requestUrl)) AS Distinct_URLs
FROM
  `ds_us.requests`
GROUP BY
  Date
ORDER BY
  Date ASC

# egress data usage by day

In [ ]:
%%bigquery --project $project_id
SELECT
  EXTRACT (DATE
  FROM
    timestamp AT TIME ZONE "Asia/Shanghai") AS Date,
  SUM(httpRequest.responseSize)/1024/1024/1024 AS response_GB
FROM
  `ds_us.requests`
#WHERE httpRequest.userAgent like '%GFE%'
GROUP BY
  Date
ORDER BY
  Date ASC

,Date,response_GB
0,2020-07-09,0.002025
1,2020-07-10,0.000114
2,2020-07-11,36.163360
3,2020-07-12,79.111459
4,2020-07-13,93.809228
5,2020-07-14,240.009002
6,2020-07-15,231.911814
7,2020-07-16,189.568747
8,2020-07-17,454.606994
9,2020-07-18,1290.357826


# egress data without cross GFE fill usage by day

In [ ]:
%%bigquery --project $project_id
SELECT
  EXTRACT (DATE
  FROM
    timestamp AT TIME ZONE "Asia/Shanghai") AS Date,
  SUM(httpRequest.responseSize)/1024/1024/1024 AS response_GB
FROM
  `ds_us.requests`
WHERE (httpRequest.userAgent not like '%Cloud-CDN-Google%') or (httpRequest.userAgent is NULL)
GROUP BY
  Date
ORDER BY
  Date ASC

,Date,response_GB
0,2020-07-09,0.002025
1,2020-07-10,0.000114
2,2020-07-11,26.610390
3,2020-07-12,62.243020
4,2020-07-13,66.499845
5,2020-07-14,64.215387


# egress data for cross GFE fill usage by day

In [ ]:
%%bigquery --project $project_id
SELECT
  EXTRACT (DATE
  FROM
    timestamp AT TIME ZONE "Asia/Shanghai") AS Date,
  SUM(httpRequest.responseSize)/1024/1024/1024 AS response_GB
FROM
  `ds_us.requests`
WHERE httpRequest.userAgent like '%Cloud-CDN-Google%'
GROUP BY
  Date
ORDER BY
  Date ASC

,Date,response_GB
0,2020-07-11,9.552970
1,2020-07-12,16.868438
2,2020-07-13,27.309383
3,2020-07-14,23.836299


# error rate by day

In [ ]:
%%bigquery --project $project_id
SELECT
  EXTRACT (DATE
  FROM
    timestamp) AS Date,
  COUNT(
  IF
    (httpRequest.status > 400,
      TRUE,
      NULL)) AS num_err,
  COUNT(*) AS sum,
  COUNT(
  IF
    (httpRequest.status >= 400,
      TRUE,
      NULL))/COUNT(*) AS percentage
FROM
  `ds_us.requests`
GROUP BY
  Date
ORDER BY
  Date

# hitRate by day

In [ ]:
%%bigquery --project $project_id
SELECT
  EXTRACT (DATE
  FROM
    timestamp) AS Date,
  count(if(httpRequest.cacheHit=true,true,null))/count(*) as hitRate
FROM
  `ds_us.requests`
WHERE httpRequest.requestUrl like '%cdn-2020-t2.wshareit.com%'
GROUP BY
  Date
ORDER BY
  Date ASC

# response bytes by userAgent

In [ ]:
%%bigquery --project $project_id
SELECT
  httpRequest.userAgent AS userAgent,
  SUM(httpRequest.responseSize)/1024/1024/1024 AS response_GB
FROM
  `ds_us.requests`
WHERE
  httpRequest.requestUrl like '%cdn-2020-t2.wshareit.com%'
GROUP BY
  httpRequest.userAgent
ORDER BY
  response_GB DESC
limit 100

# top response bytes by urls

In [ ]:
%%bigquery --project $project_id
SELECT
  httpRequest.requestUrl AS URL,
  SUM(httpRequest.responseSize)/1024/1024/1024 AS response_GB
FROM
  `ds_us.requests`
#WHERE
#  httpRequest.requestUrl like '%cdn-2020-t2.wshareit.com%'
GROUP BY
  URL
ORDER BY
  response_GB DESC
limit 50

# top response counts by urls

In [ ]:
%%bigquery --project $project_id
SELECT
  httpRequest.requestUrl as RequestURL, count(*) AS sums,
FROM
  `ds_us.requests`
#WHERE
#  httpRequest.requestUrl like '%cdn-2020-t2.wshareit.com%'
GROUP BY
  httpRequest.requestUrl
ORDER BY
  sums DESC
limit 50

# top url by day

In [ ]:
%%bigquery --project $project_id
SELECT
  httpRequest.requestUrl AS URL,
  SUM(httpRequest.responseSize)/1024/1024/1024 AS response_GB
FROM
  `ds_us.requests`
#WHERE
#  httpRequest.requestUrl like '%cdn-2020-t2.wshareit.com%'
GROUP BY
  URL
ORDER BY
  response_GB DESC
limit 50

# URL count with less than 5 requests percentage

In [ ]:
%%bigquery --project $project_id
WITH URL_COUNT as (
SELECT
  httpRequest.requestUrl AS URL,
  MAX( httpRequest.responseSize ) AS ResponseSize,
  COUNT(httpRequest.requestUrl) AS Count
FROM
  `ds_us.requests`
WHERE
  httpRequest.requestUrl like '%cdn-2020-t2.wshareit.com%'
GROUP BY
  URL
)

select 
(select count(*) from URL_COUNT where Count <= 5) as count_lt_5, 
(select count(*) from URL_COUNT) as count_total,
(select count(*) from URL_COUNT where Count <= 5)/(select count(*) from URL_COUNT) as percentage

# URL count with less than 50 requests count percentage

In [ ]:
%%bigquery --project $project_id
WITH URL_COUNT as (
SELECT
  httpRequest.requestUrl AS URL,
  MAX( httpRequest.responseSize ) AS ResponseSize,
  COUNT(httpRequest.requestUrl) AS Count
FROM
  `ds_us.requests`
WHERE
  httpRequest.requestUrl like '%cdn-2020-t2.wshareit.com%'
GROUP BY
  URL
)

select
(select sum(Count) from URL_COUNT where Count <= 50) as sum_lt_50, 
(select sum(Count) from URL_COUNT) as sum_total,
(select sum(Count) from URL_COUNT where Count <= 50)/(select sum(Count) from URL_COUNT) as percentage

,sum_lt_50,sum_total,percentage
0,87787176,1441000066,0.060921


# egress data usage by day, Bigquery SDK
## Last day and domain

In [ ]:
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

df = client.query('''
SELECT
  EXTRACT (DATE
  FROM
    timestamp AT TIME ZONE "Asia/Shanghai") AS Date,
  "cdn-2020-t2.wshareit.com" as Domain,
  SUM(httpRequest.responseSize)/1024/1024/1024 AS response_GB
FROM
  `ds_us.requests`
WHERE ((httpRequest.userAgent not like '%Cloud-CDN-Google%') or (httpRequest.userAgent is NULL))
and EXTRACT(DATE FROM timestamp AT TIME ZONE "Asia/Shanghai") = DATE_ADD(CURRENT_DATE('Asia/Shanghai'), INTERVAL -1 DAY)
and httpRequest.requestUrl like "%cdn-2020-t2.wshareit.com%"
GROUP BY
  Date
ORDER BY
  Date ASC
''' ).to_dataframe()
df.head(20)

,Date,Domain,response_GB
0,2020-07-23,cdn-2020-t2.wshareit.com,4082.560819
